# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

PEFT technique: LoRA (Low-Rank Adaptation)
Model: distilbert-base-uncased using AutoModelForSequenceClassification (binary classification)
Evaluation approach: Hugging Face Trainer with evaluate metrics (accuracy + weighted F1)
Fine-tuning dataset: imdb dataset from Hugging Face datasets library (subsampled for faster training)

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip -q install peft datasets transformers accelerate

import os, shutil
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [2]:
dataset = load_dataset("imdb")

train_ds = dataset["train"].shuffle(seed=42).select(range(2000))
test_ds  = dataset["test"].shuffle(seed=42).select(range(1000))

print("Train size:", len(train_ds), "| Test size:", len(test_ds))
print(train_ds[0]["text"][:120])

Train size: 2000 | Test size: 1000
There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. 


In [3]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

train_tok = train_ds.map(tokenize, batched=True)
test_tok  = test_ds.map(tokenize, batched=True)

train_tok = train_tok.remove_columns(["text"])
test_tok  = test_tok.remove_columns(["text"])

train_tok = train_tok.rename_column("label", "labels")
test_tok  = test_tok.rename_column("label", "labels")

train_tok.set_format("torch")
test_tok.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
base_model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = float((preds == labels).mean())

    f1s = []
    supports = []
    for cls in [0, 1]:
        tp = np.sum((preds == cls) & (labels == cls))
        fp = np.sum((preds == cls) & (labels != cls))
        fn = np.sum((preds != cls) & (labels == cls))

        precision = tp / (tp + fp + 1e-12)
        recall    = tp / (tp + fn + 1e-12)
        f1        = 2 * precision * recall / (precision + recall + 1e-12)

        support = np.sum(labels == cls)

        f1s.append(f1)
        supports.append(support)

    weighted_f1 = float((f1s[0]*supports[0] + f1s[1]*supports[1]) / (supports[0] + supports[1] + 1e-12))

    return {"accuracy": acc, "f1": weighted_f1}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
args_base = TrainingArguments(
    output_dir="/tmp/baseline_eval",
    per_device_eval_batch_size=16,
    logging_strategy="no",
    save_strategy="no",
    report_to="none"
)

trainer_base = Trainer(
    model=base_model,
    args=args_base,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

baseline_results = trainer_base.evaluate()
print("BASELINE RESULTS:", baseline_results)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


BASELINE RESULTS: {'eval_loss': 0.6969956159591675, 'eval_accuracy': 0.393, 'eval_f1': 0.3873352169419138, 'eval_runtime': 16.6264, 'eval_samples_per_second': 60.145, 'eval_steps_per_second': 3.789}


In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=["q_lin", "v_lin"]
)

peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [8]:
args_peft = TrainingArguments(
    output_dir="/tmp/peft_run",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none"
)

trainer_peft = Trainer(
    model=peft_model,
    args=args_peft,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer_peft.train()
peft_results = trainer_peft.evaluate()
print("PEFT RESULTS (after training):", peft_results)

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.290600,0.318441,0.862000,0.861895


PEFT RESULTS (after training): {'eval_loss': 0.31844109296798706, 'eval_accuracy': 0.862, 'eval_f1': 0.8618954876396948, 'eval_runtime': 17.562, 'eval_samples_per_second': 56.941, 'eval_steps_per_second': 7.118, 'epoch': 1.0}


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [9]:
peft_model.save_pretrained("/tmp/peft_adapter")
tokenizer.save_pretrained("/tmp/peft_adapter")
print("Saved files:", os.listdir("/tmp/peft_adapter"))

Saved files: ['adapter_model.bin', 'tokenizer_config.json', 'vocab.txt', 'adapter_config.json', 'README.md', 'tokenizer.json', 'special_tokens_map.json']


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [10]:
loaded_model = AutoPeftModelForSequenceClassification.from_pretrained("/tmp/peft_adapter")
loaded_model.to(device)
loaded_model.eval()

trainer_loaded = Trainer(
    model=loaded_model,
    args=args_base,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

loaded_results = trainer_loaded.evaluate()
print("RELOADED PEFT RESULTS:", loaded_results)

print("\n===== COMPARISON =====")
print("Baseline:", baseline_results)
print("Fine-tuned:", peft_results)
print("Reloaded:", loaded_results)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RELOADED PEFT RESULTS: {'eval_loss': 0.31844112277030945, 'eval_accuracy': 0.862, 'eval_f1': 0.8618954876396948, 'eval_runtime': 19.0949, 'eval_samples_per_second': 52.37, 'eval_steps_per_second': 3.299}

===== COMPARISON =====
Baseline: {'eval_loss': 0.6969956159591675, 'eval_accuracy': 0.393, 'eval_f1': 0.3873352169419138, 'eval_runtime': 16.6264, 'eval_samples_per_second': 60.145, 'eval_steps_per_second': 3.789}
Fine-tuned: {'eval_loss': 0.31844109296798706, 'eval_accuracy': 0.862, 'eval_f1': 0.8618954876396948, 'eval_runtime': 17.562, 'eval_samples_per_second': 56.941, 'eval_steps_per_second': 7.118, 'epoch': 1.0}
Reloaded: {'eval_loss': 0.31844112277030945, 'eval_accuracy': 0.862, 'eval_f1': 0.8618954876396948, 'eval_runtime': 19.0949, 'eval_samples_per_second': 52.37, 'eval_steps_per_second': 3.299}


In [11]:
dst = "./peft_adapter"
src = "/tmp/peft_adapter"

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)
print("Copied adapter to:", dst)
print("Files:", os.listdir(dst))

Copied adapter to: ./peft_adapter
Files: ['adapter_model.bin', 'tokenizer_config.json', 'vocab.txt', 'adapter_config.json', 'README.md', 'tokenizer.json', 'special_tokens_map.json']
